# 5. Tính toán Học sâu
Trong chương này, chúng tôi sẽ vén tấm màn bí ẩn và đào sâu vào những yếu tố chính của tính toán học sâu; cụ thể là việc xây dựng mô hình, truy cập và khởi tạo tham số, thiết kế các tầng và khối tùy chỉnh, đọc và ghi mô hình lên ổ cứng và cuối cùng là tận dụng GPU nhằm đạt được tốc độ đáng kể. Những hiểu biết này sẽ giúp bạn từ một *người dùng cuối* (**end user**) trở thành một *người dùng thành thạo* (**power user**), cung cấp cho bạn các công cụ cần thiết để gặt hái lợi ích của một thư viện học sâu trưởng thành, đồng thời giữ được sự linh hoạt để lập trình những mô hình phức tạp hơn, bao gồm cả những mô hình mà bạn tự phát minh! Mặc dù chương này không giới thiệu bất cứ mô hình hay tập dữ liệu mới nào, các chương sau về mô hình nâng cao sẽ phụ thuộc rất nhiều vào những kỹ thuật sắp được nhắc đến.

## 5.1. Tầng và Khối
Khi lần đầu giới thiệu về các mạng nơ-ron, ta tập trung vào các mô hình tuyến tính với một đầu ra duy nhất. Như vậy toàn bộ mô hình chỉ chứa một nơ-ron. Lưu ý rằng một nơ-ron đơn lẻ (i) nhận một vài đầu vào; (ii) tạo một đầu ra (vô hướng) tương ứng; và (iii) có một tập các tham số liên quan có thể được cập nhật để tối ưu một hàm mục tiêu nào đó mà ta quan tâm. Sau đó, khi bắt đầu nghĩ về các mạng có nhiều đầu ra, ta tận dụng các phép tính vector để mô tả nguyên một tầng nơ-ron. Cũng giống như các nơ-ron riêng lẻ, các tầng (i) nhận một số đầu vào, (ii) tạo các đầu ra tương ứng, và (iii) được mô tả bằng một tập các tham số có thể điều chỉnh được. Trong hồi quy softmax, bản thân tầng duy nhất ấy chính là một mô hình. Thậm chí đối với các perceptron đa tầng, ta vẫn có thể nghĩ về chúng theo cấu trúc cơ bản này.

Dù bạn có thể nghĩ rằng các nơ-ron, các tầng và các mô hình đã cung cấp đủ sự trừu tượng để bắt tay vào làm việc, hóa ra sẽ là thuận tiện hơn khi ta bàn về các thành phần lớn hơn một tầng riêng lẻ nhưng lại nhỏ hơn toàn bộ mô hình. Ví dụ, kiến trúc ResNet-152, rất phổ biến trong thị giác máy tính, sở hữu hàng trăm tầng. Nó bao gồm các khuôn mẫu nhóm tầng được lặp lại nhiều lần. Việc lập trình từng tầng của một mạng như vậy có thể trở nên tẻ nhạt. Mối quan tâm này không chỉ là trên lý thuyết — các khuôn mẫu thiết kế như vậy rất phổ biến trong thực tế. Kiến trúc ResNet được đề cập ở trên đã giành chiến thắng trong hai cuộc thi thị giác máy tính ImageNet và COCO năm 2015, trong cả bài toán nhận dạng và bài toán phát hiện [He et al., 2016a] và vẫn là một kiến trúc được tin dùng cho nhiều bài toán thị giác. Các kiến trúc tương tự, trong đó các tầng được sắp xếp thành những khuôn mẫu lặp lại, hiện đã trở nên thông dụng ở nhiều lĩnh vực khác, bao gồm cả xử lý ngôn ngữ tự nhiên và xử lý tiếng nói.

Để lập trình các mạng phức tạp này, ta sẽ giới thiệu khái niệm *khối* trong mạng nơ-ron. Một khối có thể mô tả một tầng duy nhất, một mảng đa tầng hoặc toàn bộ một mô hình! Dưới góc nhìn xây dựng phần mềm, một **Block** (Khối) là một lớp. Bất kỳ một lớp con nào của **Block** đều phải định nghĩa phương thức **forward** để chuyển hóa đầu vào thành đầu ra và phải lưu trữ mọi tham số cần thiết. Lưu ý rằng có một vài **Block** sẽ không yêu cầu chứa bất kỳ tham số nào cả! Ngoài ra, một **Block** phải sở hữu một phương thức **backward** cho mục đích tính toán **gradient**. May mắn thay, nhờ sự trợ giúp đắc lực của gói autograd (được giới thiệu trong Section 2) nên khi định nghĩa **Block**, ta chỉ cần quan tâm đến các tham số và hàm **forward**.

![](images/blocks.svg)

Để bắt đầu, ta sẽ xem lại các khối mà ta đã sử dụng để lập trình perceptron đa tầng (Section 4.3). Đoạn mã nguồn sau tạo ra một mạng gồm một tầng ẩn kết nối đầy đủ với 256 nút sử dụng hàm kích hoạt ReLU, theo sau là một *tầng đầu ra* kết nối đầy đủ với 10 nút (không có hàm kích hoạt).

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0466,  0.1325,  0.0032, -0.0578,  0.0310, -0.0258,  0.2492,  0.1313,
          0.0752, -0.1073],
        [-0.1251,  0.0310,  0.1532,  0.0152,  0.0380, -0.0703,  0.2163,  0.1287,
          0.0628, -0.0895]], grad_fn=<AddmmBackward>)

Trong ví dụ này, chúng tôi đã xây dựng mô hình của mình bằng cách khởi tạo một `nn.Sequential`, với các lớp theo thứ tự mà chúng sẽ được thực thi dưới dạng các đối số. Tóm lại, `nn.Sequential` định nghĩa một loại Mô-đun đặc biệt, lớp trình bày một khối trong PyTorch. Nó duy trì một danh sách có thứ tự các Mô-đun cấu thành. Lưu ý rằng mỗi lớp trong số hai lớp được kết nối đầy đủ là một thể hiện của lớp Tuyến tính, bản thân nó là một lớp con của Mô-đun. Hàm truyền tiến (chuyển tiếp) cũng rất đơn giản: nó xâu chuỗi từng khối trong danh sách với nhau, chuyển đầu ra của mỗi khối làm đầu vào cho khối tiếp theo. Lưu ý rằng cho đến nay, chúng tôi vẫn đang gọi các mô hình của mình qua `net(X)` để lấy kết quả đầu ra của chúng. Đây thực sự chỉ là cách viết tắt của `net.forward(X)`, một thủ thuật Python mà đạt được thông qua hàm `__call__` của lớp Block.

### 5.1.1. Một Khối Tùy chỉnh
Có lẽ cách dễ nhất để hiểu rõ hơn `nn.Module` hoạt động như thế nào là tự lập trình nó. Trước khi tự lập trình một Block tùy chỉnh, hãy cùng tóm tắt ngắn gọn các chức năng cơ bản mà một Block phải cung cấp:

1. Phương thức forward nhận đối số là dữ liệu đầu vào.
2. Phương thức forward trả về một giá trị đầu ra. Lưu ý rằng đầu ra có thể có kích thước khác với đầu vào. Ví dụ, tầng Dense đầu tiên trong mô hình phía trên nhận đầu vào có kích thước tùy ý nhưng trả về đầu ra có kích thước 256.
3. Tính gradient của đầu ra theo đầu vào bằng phương thức backward, thường thì việc này được thực hiện tự động.
4. Lưu trữ và cung cấp quyền truy cập tới các tham số cần thiết để tiến hành phương thức tính toán forward.
5. Khởi tạo các tham số này khi cần thiết.

Trong đoạn mã dưới đây, chúng ta lập trình từ đầu một Block (Khối) tương đương với một perceptron đa tầng chỉ có một tầng ẩn và 256 nút ẩn, cùng một tầng đầu ra 10 chiều. Lưu ý rằng lớp MLP bên dưới đây kế thừa từ lớp Block. Ta sẽ phụ thuộc nhiều vào các phương thức của lớp cha, và chỉ tự viết phương thức `__init__` và forward.

In [2]:
class MLP(nn.Module):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Block` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # Hidden layer
        self.out = nn.Linear(256, 10)  # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def forward(self, X):
        # Note here we use the funtional version of ReLU defined in the
        # nn.functional module.
        return self.out(F.relu(self.hidden(X)))

Để bắt đầu, ta sẽ tập trung vào phương thức forward. Lưu ý rằng nó nhận giá trị đầu vào x, tính toán tầng biểu diễn ẩn và trả về các giá trị logit. Ở cách lập trình MLP này, cả hai tầng trên đều là biến thực thể (instance variables). Để thấy tại sao điều này có lý, tưởng tượng ta khởi tạo hai MLP, net1 và net2, và huấn luyện chúng với dữ liệu khác nhau. Dĩ nhiên là ta mong đợi chúng đại diện cho hai mô hình học khác nhau.

Ta khởi tạo các tầng của MLP trong phương thức `__init__` (hàm khởi tạo) và sau đó gọi các tầng này mỗi khi ta gọi phương thức forward. Hãy chú ý một vài chi tiết quan trọng. Đầu tiên, phương thức `__init__` tùy chỉnh của ta gọi phương thức `__init__` của lớp cha thông qua `super().__init__` để tránh việc viết lại cùng một phần mã nguồn áp dụng cho hầu hết các khối. Chúng ta sau đó khởi tạo hai tầng, gán chúng lần lượt là `self.hidden` và `self.output`. Chú ý rằng trừ khi đang phát triển một toán tử mới, chúng ta không cần lo lắng về lan truyền ngược (phương thức backward) hoặc khởi tạo tham số (phương thức initialize).

In [3]:
net = MLP()
net(X)

tensor([[-0.0641,  0.0769, -0.1082, -0.1626, -0.0957, -0.1632,  0.1201, -0.1002,
         -0.0150,  0.1364],
        [-0.0349,  0.1103, -0.1302, -0.1210,  0.0073, -0.0490, -0.0030, -0.1136,
          0.0523,  0.2368]], grad_fn=<AddmmBackward>)

### 5.1.2. Khối Tuần tự
Bây giờ ta có thể có cái nhìn rõ hơn về cách mà lớp Sequential (Tuần tự) hoạt động. Nhắc lại rằng Sequential được thiết kế để xâu chuỗi các Khối lại với nhau. Để tự xây dựng một lớp MySequential đơn giản, ta chỉ cần định nghĩa hai phương thức chính sau: 1. Phương thức nối thêm nhằm đẩy từng Block một vào trong danh sách. 2. Phương thức forward nhằm truyền một đầu vào qua chuỗi các Blocks (theo thứ tự mà chúng được nối).

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # Here, `block` is an instance of a `Module` subclass. We save it
            # in the member variable `_modules` of the `Module` class, and its
            # type is OrderedDict
            self._modules[block] = block

    def forward(self, X):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        for block in self._modules.values():
            X = block(X)
        return X

Phương thức `__int__` thêm một Block đơn vào từ điển có thứ tự `_modules`. Bạn có thể thắc mắc tại sao mỗi Block sở hữu một thuộc tính `_modules` và tại sao ta sử dụng nó thay vì tự tạo một danh sách Python. Thật ra, ưu điểm chính của `_modules` là trong quá trình khởi tạo trọng số ban đầu của các khối, sẽ tự động tìm các khối con có trọng số cần được khởi tạo trong từ điển này.

Khi phương thức forward của khối `MySequential` được gọi, các Block sẽ được thực thi theo thứ tự mà chúng được thêm vào. Bây giờ ta có thể lập trình lại một MLP sử dụng lớp `MySequential`.

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1972, -0.1056,  0.2349, -0.2546,  0.0506,  0.0771, -0.3776,  0.0589,
          0.0783, -0.0044],
        [-0.1769, -0.1699,  0.3765, -0.1693, -0.0061, -0.0063, -0.3469,  0.0629,
          0.0562,  0.0010]], grad_fn=<AddmmBackward>)

### 5.1.3. Thực thi Mã trong Phương thức forward
Lớp Sequential giúp việc xây dựng mô hình trở nên dễ hơn, cho phép ta xây dựng các kiến trúc mới mà không cần phải tự định nghĩa một lớp riêng. Tuy nhiên, không phải tất cả mô hình đều có cấu trúc chuỗi xích đơn giản. Khi cần phải linh hoạt hơn, ta vẫn sẽ muốn định nghĩa từng Block theo cách của mình, ví dụ như khi muốn sử dụng luồng điều khiển Python trong lượt truyền xuôi. Hơn nữa, ta cũng có thể muốn thực hiện các phép toán tùy ý thay vì chỉ dựa vào các tầng mạng nơ-ron được định nghĩa từ trước.

Độc giả có thể nhận ra rằng tất cả phép toán trong mạng cho tới giờ đều thao tác trên các giá trị kích hoạt và tham số của mạng. Tuy nhiên, trong một vài trường hợp, ta có thể muốn kết hợp thêm các hằng số. Chúng không phải là kết quả của tầng trước mà cũng không phải là tham số có thể cập nhật được. Ta gọi chúng là tham số không đổi (constant parameter). Ví dụ ta muốn một tầng tính hàm $f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$ , trong đó  $x ,  w$  là tham số, và  $c$  là một hằng số cho trước được giữ nguyên giá trị trong suốt quá trình tối ưu hóa.

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # Use the created constant parameters, as well as the `relu` and `mm`
        # functions
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

Trong mô hình FixedHiddenMLP, ta lập trình một tầng ẩn có trọng số (self.rand_ weight) được khởi tạo ngẫu nhiên và giữ nguyên giá trị về sau. Trọng số này không phải là một tham số mô hình, vì vậy nó không được cập nhật khi sử dụng lan truyền ngược. Sau đó, đầu ra của tầng cố định này được đưa vào tầng kết nối đầy đủ (fully-connected layer).

Lưu ý rằng trước khi trả về giá trị đầu ra, mô hình của ta đã làm điều gì đó bất thường. Ta đã chạy một vòng lặp `while`, lấy vector đầu ra chia cho  $2$  cho đến khi nó thỏa mãn điều kiện `abs(x).sum() > 1`. Cuối cùng, ta gán giá trị đầu ra bằng tổng các phần tử trong x. Theo sự hiểu biết của chúng tôi, không có mạng nơ-ron tiêu chuẩn nào thực hiện phép toán này. Lưu ý rằng phép toán đặc biệt này có thể không hữu ích gì trong các công việc ngoài thực tế. Mục đích của chúng tôi ở đây là chỉ cho độc giả thấy được cách tích hợp một đoạn mã tùy ý vào luồng tính toán của mạng nơ-ron.

In [7]:
net = FixedHiddenMLP()
net(X)

tensor(0.2913, grad_fn=<SumBackward0>)

Ta có thể kết hợp nhiều cách khác nhau để lắp ráp các Block lại. Trong ví dụ dưới đây, ta lồng các Block với nhau theo nhiều cách sáng tạo.

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0146, grad_fn=<SumBackward0>)

### 5.1.4. Biên dịch Mã nguồn¶
Những người đọc có tâm có thể sẽ bắt đầu lo lắng về hiệu năng của một vài đoạn mã trên. Sau cùng thì, chúng ta có rất nhiều thao tác truy cập từ điển, thực thi mã lập trình và rất nhiều thứ “đậm chất Python” khác xuất hiện trong thứ mà lẽ ra nên là một thư viện học sâu hiệu năng cao. Vấn đề của Khóa Trình thông dịch Toàn cục ([Global Interpreter Lock](https://wiki.python.org/moin/GlobalInterpreterLock)) trong Python khá phổ biến. Trong bối cảnh học sâu, ta lo sợ rằng GPU cực kỳ nhanh của ta có thể sẽ phải đợi CPU “rùa bò” chạy xong những dòng lệnh Python trước khi nó có thể nhận tác vụ chạy tiếp theo. Cách tốt nhất để tăng tốc Python là tránh không sử dụng nó.

### 5.1.5. Tóm tắt
* Các tầng trong mạng nơ-ron là các Khối.
* Nhiều tầng có thể cấu thành một Khối.
* Nhiều Khối có thể cấu thành một Khối.
* Một Khối có thể chứa các đoạn mã nguồn.
* Các Khối đảm nhiệm nhiều tác vụ bao gồm khởi tạo tham số và lan truyền ngược.
* Việc gắn kết các tầng và khối một cách tuần tự được đảm nhiệm bởi Khối Sequential.

### 5.1.6. Bài tập
1. Những loại vấn đề nào sẽ xảy ra nếu bạn thay đổi `MySequential` để lưu trữ các khối trong danh sách Python?
2. Hãy lập trình một khối nhận đối số là hai khối khác, ví dụ như `net1` và `net2`, và trả về kết quả là phép nối các giá trị đầu ra của cả hai mạng đó khi thực hiện lượt truyền xuôi.
3. Giả sử bạn muốn nối nhiều thực thể của cùng một mạng với nhau. Hãy lập trình một hàm để tạo ra nhiều thực thể của cùng một mạng và dùng chúng để tạo thành một mạng lớn hơn (các hàm này trong thiết kế phần mềm được gọi là Factory Function).

## 5.2 Quản lý Tham số

Một khi ta đã chọn được kiến trúc mạng và các giá trị siêu tham số, ta sẽ bắt đầu với vòng lặp huấn luyện với mục tiêu là tìm các giá trị tham số để cực tiểu hóa hàm mục tiêu. Sau khi huấn luyện xong, ta sẽ cần các tham số đó để đưa ra dự đoán trong tương lai. Hơn nữa, thi thoảng ta sẽ muốn trích xuất tham số để sử dụng lại trong một hoàn cảnh khác, có thể lưu trữ mô hình để thực thi trong một phần mềm khác hoặc để rút ra hiểu biết khoa học bằng việc phân tích mô hình.

Thông thường, ta có thể bỏ qua những chi tiết chuyên sâu về việc khai báo và xử lý tham số bởi deep learning framework sẽ đảm nhiệm công việc nặng nhọc này. Tuy nhiên, khi ta bắt đầu tiến xa hơn những kiến trúc chỉ gồm các tầng tiêu chuẩn được xếp chồng lên nhau, đôi khi ta sẽ phải tự đi sâu vào việc khai báo và xử lý tham số. Trong mục này, chúng tôi sẽ đề cập đến những việc sau:

   * Truy cập các tham số để gỡ lỗi, chẩn đoán mô hình và biểu diễn trực quan.
   * Khởi tạo tham số.
   * Chia sẻ tham số giữa các thành phần khác nhau của mô hình.

Chúng ta sẽ bắt đầu từ mạng Perceptron đa tầng với một tầng ẩn.

In [9]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.0700],
        [-0.1383]], grad_fn=<AddmmBackward>)

### 5.2.1. Truy cập Tham số
Hãy bắt đầu với việc truy cập tham số của những mô hình mà bạn đã biết. Khi một mô hình được định nghĩa bằng lớp Tuần tự (Sequential), ta có thể truy cập bất kỳ tầng nào bằng chỉ số, như thể nó là một danh sách. Thuộc tính params của mỗi tầng chứa tham số của chúng. Ta có thể quan sát các tham số của mạng net định nghĩa ở trên.

In [10]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.2234, -0.2544, -0.0499,  0.2056, -0.3039,  0.1725,  0.2018,  0.2801]])), ('bias', tensor([0.0007]))])


Kết quả của đoạn mã này cho ta một vài thông tin quan trọng. Đầu tiên, mỗi tầng kết nối đầy đủ đều có hai tập tham số, như `weight` và `bias` tương ứng với trọng số và hệ số điều chỉnh của tầng đó. Chúng đều được lưu trữ ở dạng số thực dấu phẩy động độ chính xác đơn. Lưu ý rằng tên của các tham số cho phép ta xác định tham số của từng tầng một cách độc nhất, kể cả khi mạng nơ-ron chứa hàng trăm tầng.

#### 5.2.1.1. Các tham số Mục tiêu
Lưu ý rằng mỗi tham số được biểu diễn bằng một thực thể của lớp Parameter. Để làm việc với các tham số, trước hết ta phải truy cập được các giá trị số của chúng. Có một vài cách để làm việc này, một số cách đơn giản hơn trong khi các cách khác lại tổng quát hơn. Để bắt đầu, ta có thể truy cập tham số của một tầng thông qua thuộc tính `bias` hoặc `weight` rồi sau đó truy cập giá trị số của chúng thông qua phương thức `data()`. Đoạn mã sau trích xuất hệ số điều chỉnh của tầng thứ hai trong mạng nơ-ron.

In [11]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0007], requires_grad=True)
tensor([0.0007])


Tham số là các đối tượng khá phức tạp bởi chúng chứa dữ liệu, gradient và một vài thông tin khác. Đó là lý do tại sao ta cần yêu cầu dữ liệu một cách tường minh.
Ngoài giá trị, mỗi tham số còn cho phép chúng ta truy cập vào gradient. Bởi vì chúng tôi chưa gọi backpropagation cho mạng này, nó đang ở trạng thái ban đầu.

In [12]:
net[2].weight.grad == None

True

#### 5.2.1.2. Tất cả các Tham số cùng lúc
Khi ta cần phải thực hiện các phép toán với tất cả tham số, việc truy cập lần lượt từng tham số sẽ trở nên khá khó chịu. Việc này sẽ càng chậm chạp khi ta làm việc với các khối phức tạp hơn, ví dụ như các khối lồng nhau vì lúc đó ta sẽ phải duyệt toàn bộ cây bằng đệ quy để có thể trích xuất tham số của từng khối con. Dưới đây, chúng tôi chứng minh việc truy cập các tham số của lớp đầu tiên được kết nối đầy đủ so với truy cập tất cả các lớp.

In [13]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


Từ đó, ta có cách thứ ba để truy cập các tham số của mạng:

In [14]:
net.state_dict()['2.bias'].data

tensor([0.0007])

#### 5.2.1.3. Thu thập Tham số từ các Khối lồng nhau
Hãy cùng xem cách hoạt động của các quy ước định danh tham số khi ta lồng nhiều khối vào nhau. Trước hết ta định nghĩa một hàm tạo khối (có thể gọi là một nhà máy khối) và rồi kết hợp chúng trong các khối lớn hơn.

In [15]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # Nested here
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.5780],
        [-0.5780]], grad_fn=<AddmmBackward>)

Bây giờ ta đã xong phần thiết kế mạng, hãy cùng xem cách nó được tổ chức.

In [16]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


Bởi vì các tầng được lồng vào nhau theo cơ chế phân cấp, ta cũng có thể truy cập chúng tương tự như cách ta dùng chỉ số để truy cập các danh sách lồng nhau. Chẳng hạn, ta có thể truy cập khối chính đầu tiên, khối con thứ hai bên trong nó và hệ số điều chỉnh của tầng đầu tiên bên trong nữa như sau:

In [17]:
rgnet[0][1][0].bias.data

tensor([ 0.0297,  0.3724,  0.4030,  0.0732, -0.1647, -0.0619,  0.2648,  0.3776])

### 5.2.2. Khởi tạo Tham số
Bây giờ khi đã biết cách truy cập tham số, hãy cùng xem xét việc khởi tạo chúng đúng cách. Ta đã thảo luận về sự cần thiết của việc khởi tạo tham số trong Section 4.8. Deep learning framework cung cấp các khởi tạo ngẫu nhiên mặc định cho các lớp. Tuy nhiên, thường ta sẽ muốn khởi tạo trọng số theo nhiều phương pháp khác.Các framwork cung cấp sẵn các phương thức khởi tạo. Nếu ta muốn một bộ khởi tạo tùy chỉnh, ta sẽ cần làm thêm một chút việc.

Theo mặc định, PyTorch khởi tạo ma trận trọng số và độ lệch một cách đồng nhất bằng cách khởi tạo từ một phạm vi được tính theo thứ nguyên đầu vào và đầu ra. Mô-đun PyTorch’s `nn.init` cung cấp nhiều phương pháp khởi tạo đặt trước.

#### 5.2.2.1. Phương thức Khởi tạo có sẵn
Ta sẽ bắt đầu với việc gọi các bộ khởi tạo có sẵn. Đoạn mã dưới đây khởi tạo tất cả các tham số với các biến ngẫu nhiên Gauss có độ lệch chuẩn bằng 0.01.

In [18]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0008, -0.0008, -0.0033,  0.0093]), tensor(0.))

Ta cũng có thể khởi tạo tất cả tham số với một hằng số (ví dụ như 1) bằng cách sử dụng bộ khởi tạo Constant.

In [19]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

Ta còn có thể áp dụng các bộ khởi tạo khác nhau cho các khối khác nhau. Ví dụ, trong đoạn mã nguồn bên dưới, ta khởi tạo tầng đầu tiên bằng cách sử dụng bộ khởi tạo Xavier và khởi tạo tầng thứ hai với một hằng số là 42.

In [20]:
def xavier(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        torch.nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.1738, -0.1166,  0.4947,  0.5316])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


#### 5.2.2.2. Phương thức Khởi tạo Tùy chỉnh
Đôi khi, các phương thức khởi tạo mà ta cần không có sẵn trong deep learning framework. Trong ví dụ dưới đây, chúng tôi xác định bộ khởi tạo cho bất kỳ tham số trọng lượng w nào bằng cách sử dụng phân phối lạ sau:

\begin{split}\begin{aligned}
    w \sim \begin{cases}
        U[5, 10] & \text{ với xác suất } \frac{1}{4} \\
            0    & \text{ với xác suất } \frac{1}{2} \\
        U[-10, -5] & \text{ với xác suất } \frac{1}{4}
    \end{cases}
\end{aligned}\end{split}

In [21]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]


Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  0.0000, -0.0000, -0.0000],
        [ 0.0000, -0.0000,  0.0000, -6.5115]], grad_fn=<SliceBackward>)

Lưu ý rằng ta luôn có thể trực tiếp đặt giá trị cho tham số bằng cách gọi hàm `data()`

In [22]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.,  1.,  1.,  1.])

### 5.2.3. Các Tham số bị Trói buộc
Thông thường, ta sẽ muốn chia sẻ các tham số mô hình cho nhiều tầng. Sau này ta sẽ thấy trong quá trình huấn luyện embedding từ, việc sử dụng cùng một bộ tham số để mã hóa và giải mã các từ có thể khá hợp lý. Sau đây ta sẽ tạo một tầng kết nối đầy đủ và sử dụng chính tham số của nó làm tham số cho một tầng khác.

In [23]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


Ví dụ này cho thấy các tham số của tầng thứ hai và thứ ba đã bị trói buộc với nhau. Chúng không chỉ có giá trị bằng nhau, chúng còn được biểu diễn bởi cùng một ndarray. Vì vậy, nếu ta thay đổi các tham số của tầng này này thì các tham số của tầng kia cũng sẽ thay đổi theo. Bạn có thể tự hỏi rằng *chuyện gì sẽ xảy ra với gradient khi các tham số bị trói buộc?*. Vì các tham số mô hình chứa gradient nên gradient của tầng ẩn thứ hai và tầng ẩn thứ ba được cộng lại tại trong quá trình lan truyền ngược.

### 5.2.4. Tóm tắt

   * Ta có vài cách để truy cập, khởi tạo và trói buộc các tham số mô hình.
   * Ta có thể sử dụng các phương thức khởi tạo tùy chỉnh.
   * Gluon có một cơ chế tinh vi để truy cập các tham số theo phân cấp một cách độc nhất.

### 5.2.5. Bài tập
1. Sử dụng FancyMLP được định nghĩa trong Section 5.1 và truy cập tham số của các tầng khác nhau.
2. Xem tài liệu mô-đun khởi tạo để khám phá các trình khởi tạo khác nhau.
3. Xây dựng một MLP chứa một lớp tham số dùng chung và huấn luyện nó. Trong quá trình đào tạo, hãy quan sát các thông số mô hình và độ dốc của từng lớp.
4. Why is sharing parameters a good idea?

## 5.3. Khởi tạo trễ
Cho tới nay, có vẻ như ta chưa phải chịu hậu quả của việc thiết lập mạng cẩu thả. Cụ thể, ta đã “giả mù” và làm những điều không trực quan sau:

   * Ta định nghĩa kiến trúc mạng mà không xét đến chiều đầu vào.
   * Ta thêm các tầng mà không xét đến chiều đầu ra của tầng trước đó.
   * Ta thậm chí còn “khởi tạo” các tham số mà không có đầy đủ thông tin để xác định số lượng các tham số của mô hình.
   
Bạn có thể khá bất ngờ khi thấy mã nguồn của ta vẫn chạy. Suy cho cùng, các framework không thể dự đoán được chiều của đầu vào. Thủ thuật ở đây đó là đã “khởi tạo trễ *(defers initialization)*”, tức đợi cho đến khi ta truyền dữ liệu qua mô hình lần đầu để suy ra kích thước của mỗi tầng khi chúng “di chuyển”.

Ở các chương sau, khi làm việc với các mạng nơ-ron tích chập, kỹ thuật này sẽ còn trở nên tiện lợi hơn, bởi chiều của đầu vào (tức độ phân giải của một bức ảnh) sẽ tác động đến chiều của các tầng tiếp theo trong mạng. Do đó, khả năng gán giá trị các tham số mà không cần biết số chiều tại thời điểm viết mã có thể đơn giản hóa việc xác định và sửa đổi mô hình về sau một cách đáng kể. Tiếp theo, chúng ta sẽ đi sâu hơn vào cơ chế của việc khởi tạo.

### 5.3.1. Khởi tạo Mạng
Để bắt đầu, hãy cùng khởi tạo một MLP.

In [24]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

Lúc này, mạng nơ-ron không thể biết được chiều của các trọng số ở tầng đầu vào bởi nó còn chưa biết chiều của đầu vào. Và vì thế ta chưa khởi tạo bất kỳ tham số nào cả. Ta có thể xác thực việc này bằng cách truy cập các tham số như dưới đây.

In [25]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

Lưu ý rằng mỗi đối tượng lớp tồn tại nhưng trọng số trống. Sử dụng `net.get_weights()` sẽ gây ra lỗi vì các trọng số chưa được khởi tạo. Tiếp theo, chúng ta hãy truyền dữ liệu qua mạng để làm cho framework khởi tạo các tham số cuối cùng.

In [26]:
X = tf.random.uniform((2, 20))
net(X)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]

Như ta đã thấy, không có gì thay đổi ở đây cả. Khi chưa biết chiều của đầu vào, việc gọi phương thức khởi tạo không thực sự khởi tạo các tham số. Thay vào đó, việc gọi phương thức trên sẽ chỉ đăng ký với framework là chúng ta muốn khởi tạo các tham số và phân phối mà ta muốn dùng để khởi tạo (không bắt buộc). Chỉ khi truyền dữ liệu qua mạng thì framework mới khởi tạo các tham số và ta mới thấy được sự khác biệt.

### 5.3.2. Tóm tắt

   * Khởi tạo trễ có thể khá tiện lợi, cho phép framework suy ra kích thước của tham số một cách tự động và nhờ vậy giúp ta dễ dàng sửa đổi các kiến trúc mạng cũng như loại bỏ những nguồn gây lỗi thông dụng.
   * Chúng ta không cần khởi tạo trễ khi đã định nghĩa các biến một cách tường minh.
   * Chúng ta có thể cưỡng chế việc khởi tạo lại các tham số mạng bằng cách gọi khởi tạo với `force_reinit=True`.


### 5.3.3. Bài tập

1. Chuyện gì xảy ra nếu ta chỉ chỉ rõ chiều đầu vào của tầng đầu tiên nhưng không làm vậy với các tầng tiếp theo? Việc khởi tạo có xảy ra ngay lập tức không?
2. Chuyện gì xảy ra nếu ta chỉ định các chiều không khớp nhau?
3. Bạn cần làm gì nếu đầu vào có chiều biến thiên? Gợi ý - hãy tìm hiểu về cách ràng buộc tham số (parameter tying).


## 5.4. Các tầng Tuỳ chỉnh

Một trong những yếu tố dẫn đến thành công của học sâu là sự đa dạng của các tầng. Những tầng này có thể được sắp xếp theo nhiều cách sáng tạo để thiết kế nên những kiến trúc phù hợp với nhiều tác vụ khác nhau. Ví dụ, các nhà nghiên cứu đã phát minh ra các tầng chuyên dụng để xử lý ảnh, chữ viết, lặp trên dữ liệu tuần tự, thực thi quy hoạch động, v.v… Dù sớm hay muộn, bạn cũng sẽ gặp (hoặc sáng tạo) một tầng không có trong framework. Đối với những trường hợp như vậy, bạn cần xây dựng một tầng tuỳ chỉnh. Phần này sẽ hướng dẫn bạn cách thực hiện điều đó.

### 5.4.1. Các tầng không có Tham số¶

Để bắt đầu, ta tạo một tầng tùy chỉnh (một Khối) không chứa bất kỳ tham số nào. Bước này khá quen thuộc nếu bạn còn nhớ phần giới thiệu về Block tại Section 5.1. Lớp `CenteredLayer` chỉ đơn thuần trừ đi giá trị trung bình từ đầu vào của nó. Để xây dựng nó, chúng ta chỉ cần kế thừa từ lớp `Block` và lập trình phương thức forward.

In [27]:
import torch
from torch import nn
import torch.nn.functional as F

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

Hãy cùng xác thực rằng tầng này hoạt động như ta mong muốn bằng cách truyền dữ liệu vào nó.

In [28]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

Chúng ta cũng có thể kết hợp tầng này như là một thành phần để xây dựng các mô hình phức tạp hơn.

In [29]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Để kiểm tra thêm, chúng ta có thể truyền dữ liệu ngẫu nhiên qua mạng và chứng thực xem giá trị trung bình đã về 0 hay chưa. Chú ý rằng vì đang làm việc với các số thực dấu phẩy động, chúng ta sẽ thấy một giá trị khác không rất nhỏ.

In [30]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(1.8626e-09, grad_fn=<MeanBackward0>)

### 5.4.2. Tầng có Tham số
Bây giờ chúng ta đã biết cách xác định các lớp đơn giản, chúng ta hãy chuyển sang định nghĩa các lớp với các tham số có thể được điều chỉnh thông qua training. Chúng ta có thể sử dụng các hàm tích hợp để tạo các tham số, cung cấp một số chức năng quản lý cơ bản. Đặc biệt, chúng điều chỉnh các tham số mô hình truy cập, khởi tạo, chia sẻ, lưu và tải. Bằng cách này, trong số các lợi ích khác, chúng tôi sẽ không cần phải viết các quy trình tuần tự hóa tùy chỉnh cho mọi lớp tùy chỉnh.

Bây giờ chúng ta hãy triển khai lớp được kết nối đầy đủ của riêng mình. Nhớ lại rằng lớp này yêu cầu hai tham số, một để đại diện cho trọng số và một cho độ lệch. Trong việc triển khai này, chúng tôi đặt kích hoạt ReLU làm mặc định. Lớp này yêu cầu nhập các đối số: `in_units` và `unit`, biểu thị số lượng đầu vào và đầu ra tương ứng.

In [31]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

Tiếp theo, chúng tôi khởi tạo lớp MyDense và truy cập các tham số mô hình.

In [32]:
dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[-0.0777,  1.7946,  0.3822],
        [-0.3575, -0.0605,  0.7253],
        [-0.0958, -0.4133, -1.8587],
        [-0.8156, -1.1369, -0.1198],
        [ 2.2947,  0.3520,  2.4369]], requires_grad=True)

Chúng tôi có thể trực tiếp thực hiện các phép tính truyền chuyển tiếp bằng cách sử dụng các lớp tùy chỉnh.

In [33]:
dense(torch.rand(2, 5))

tensor([[0.3506, 1.0110, 0.3021],
        [0.0000, 0.0000, 0.0000]])

Chúng tôi cũng có thể xây dựng mô hình bằng cách sử dụng các lớp tùy chỉnh. Khi chúng ta có nó, chúng ta có thể sử dụng nó giống như lớp được kết nối đầy đủ được tích hợp sẵn.

In [34]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[ 8.8238],
        [18.6645]])

### 5.4.3 Tóm tắt
* Chúng ta có thể thiết kế các lớp tùy chỉnh thông qua lớp lớp cơ bản. Điều này cho phép chúng tôi xác định các lớp mới linh hoạt hoạt động khác với bất kỳ lớp nào hiện có trong thư viện.
* Sau khi được xác định, các lớp tùy chỉnh có thể được gọi trong các ngữ cảnh và kiến trúc tùy ý.
* Các lớp có thể có các tham số cục bộ, có thể được tạo thông qua các hàm tích hợp.

### 5.4.4 Bài tập
1. Thiết kế một tầng nhận đầu vào và tính toán phép giảm tensor, tức trả về $y_k = \sum_{i, j} W_{ijk} x_i x_j$.
2. Thiết kế một tầng trả về nửa đầu của các hệ số Fourier của dữ liệu.

## 5.5. Đọc/Ghi tệp

Đến nay, ta đã thảo luận về cách xử lý dữ liệu và cách xây dựng, huấn luyện, kiểm tra những mô hình học sâu. Tuy nhiên, có thể đến một lúc nào đó ta sẽ cảm thấy hài lòng với những gì thu được và muốn lưu lại kết quả để sau này sử dụng trong những bối cảnh khác nhau (thậm chí có thể đưa ra dự đoán khi triển khai). Ngoài ra, khi vận hành một quá trình huấn luyện dài hơi, cách tốt nhất là lưu kết quả trung gian một cách định kỳ (điểm kiểm tra) nhằm đảm bảo rằng ta sẽ không mất kết quả tính toán sau nhiều ngày nếu chẳng may ta vấp phải dây nguồn của máy chủ. Vì vậy, đã đến lúc chúng ta học cách đọc và lưu trữ đồng thời các vector trọng số riêng lẻ cùng toàn bộ các mô hình. Mục này sẽ giải quyết cả hai vấn đề trên.

### 5.5.1. Đọc và Lưu các Tensor
Đối với tensor riêng lẻ, ta có thể sử dụng trực tiếp các hàm `load` và `save` để đọc và ghi tương ứng. Cả hai hàm đều yêu cầu ta cung cấp tên, và hàm save yêu cầu đầu vào với biến đã được lưu.

In [37]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

Bây giờ, chúng ta có thể đọc lại dữ liệu từ các tệp được lưu vào trong bộ nhớ.

In [38]:
x2 = torch.load("x-file")
x2

tensor([0, 1, 2, 3])

Chúng ta có thể lưu danh sách các tensor và đọc lại chúng vào trong bộ nhớ.

In [39]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

Ta còn có thể ghi và đọc một từ điển ánh xạ từ một chuỗi sang một tensor. Điều này khá là thuận tiện khi chúng ta muốn đọc hoặc ghi tất cả các trọng số của một mô hình.

In [40]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

### 5.5.2 Đọc và ghi tham số mô hình
Khả năng lưu từng vector trọng số đơn lẻ (hoặc các ndarray tensor khác) là hữu ích nhưng sẽ mất nhiều thời gian nếu chúng ta muốn lưu (và sau đó nạp lại) toàn bộ mô hình. Dù sao, chúng ta có thể có hàng trăm nhóm tham số rải rác xuyên suốt mô hình. Vì lý do đó mà framework cung cấp sẵn tính năng lưu và nạp toàn bộ các mạng. Một chi tiết quan trọng cần lưu ý là chức năng này chỉ lưu các tham số của mô hình, không phải là toàn bộ mô hình. Điều đó có nghĩa là nếu ta có một MLP ba tầng, ta cần chỉ rõ kiến trúc này một cách riêng lẻ. Lý do là vì bản thân các mô hình có thể chứa mã nguồn bất kỳ, chúng không được thêm vào tập tin một cách dễ dàng như các tham số. Vì vậy, để khôi phục lại một mô hình thì chúng ta cần xây dựng kiến trúc của nó từ mã nguồn rồi nạp các tham số từ ổ cứng vào kiến trúc này. Việc khởi tạo trễ (Section 5.3) lúc này rất có lợi vì ta chỉ cần định nghĩa một mô hình mà không cần gán giá trị cụ thể cho tham số. Như thường lệ, hãy bắt đầu với một MLP quen thuộc.

In [41]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

Tiếp theo, chúng tôi lưu trữ các tham số của mô hình dưới dạng tệp có tên “mlp.params”.

In [42]:
torch.save(net.state_dict(), 'mlp.params')

Để khôi phục mô hình, chúng ta tạo một đối tượng khác dựa trên mô hình MLP gốc. Thay vì khởi tạo ngẫu nhiên những tham số mô hình, ta đọc các tham số được lưu trực tiếp trong tập tin. 

In [44]:
clone = MLP()
clone.load_state_dict(torch.load("mlp.params"))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

Vì cả hai đối tượng của mô hình có cùng bộ tham số, kết quả tính toán với cùng đầu vào x sẽ như nhau. Hãy kiểm chứng điều này.

In [46]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

### 5.5.3. Tóm tắt

* Hàm save và load có thể được sử dụng để thực hiện việc xuất nhập tập tin cho các đối tượng ndarray.
* Chúng ta có thể lưu toàn bộ tập tham số của một mạng.
* Việc lưu kiến trúc này phải được hoàn thiện trong chương trình thay vì trong các tham số.

### 5.5.4. Bài tập¶

1. Nếu không cần phải triển khai các mô hình huấn luyện sang một thiết bị khác, theo bạn thì lợi ích thực tế của việc lưu các tham số mô hình là gì?
2. Giả sử chúng ta muốn sử dụng lại chỉ một phần của một mạng nào đó để phối hợp với một mạng của một kiến trúc khác. Trong trường hợp ta muốn sử dụng hai lớp đầu tiên của mạng trước đó vào trong một mạng mới, bạn sẽ làm thể nào để thực hiện được việc này?
3. Làm thế nào để bạn có thể lưu kiến trúc mạng và các tham số? Có những hạn chế nào khi bạn tận dụng kiến trúc này?